In [1]:

import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import scipy.io as sciio
from time import time
import sklearn
# %matplotlib inline
from sklearn.datasets import make_blobs
from sklearn.decomposition import NMF
from sklearn.svm import SVC
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import skimage
from sklearn import svm, metrics, datasets
from sklearn.svm import SVC
from sklearn.utils import Bunch
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score

from skimage.io import imread
from skimage.transform import resize
from skimage.transform import resize as imresize

REAL_IMAG = True
NO_NMF = False

In [2]:
# load data
# def Angle(data):
#     ans = []
#     for i in data:
#         # x=np.arctan(np.imag(i)/np.real(i))
#         # ans.append(np.arctan(np.imag(i)/np.real(i)))
#         ans.append(np.angle(i))
#     return ans


# Load images in structured directory like it's sklearn sample dataset
def load_image_files(container_path, dimension=(64, 64)):  # 调整图片的尺寸为dimension=(64, 64)

    image_dir = Path(container_path)
    # folders is the list of folders each conains a category of data
    folders = [directory for directory in image_dir.iterdir() if directory.is_dir()]
    # _______________________________________________________
    # 去掉annotations文件夹
    # folders = folders[:-1]
    # _______________________________________________________
    categories = [fo.name for fo in folders]

    descr = "A image classification dataset"
    images = []
    flat_data = []
    target = []

    for i, direc in enumerate(folders):
        # print(i)
        print(i, categories[i])
        # print(direc)
        for file in direc.iterdir():
            mat_data = sciio.loadmat(file)
            # img = np.abs(mat_data['frame_Ev'])
            raw_data = []
            if REAL_IMAG:
                # raw_data.extend(np.real(mat_data['frame_Ev']).flatten())
                # raw_data.extend(np.imag(mat_data['frame_Ev']).flatten())
                # raw_data.extend(np.real(mat_data['frame_Eh']).flatten())
                # raw_data.extend(np.imag(mat_data['frame_Eh']).flatten())
                # raw_data = np.abs(raw_data)
                raw_data.extend(np.abs(mat_data['frame_Ev']).flatten())
                # raw_data.extend((np.angle(mat_data['frame_Ev'].flatten())))
                raw_data.extend(((np.angle(mat_data['frame_Ev'].flatten()) / np.pi + 1) * 100))
                raw_data.extend(np.abs(mat_data['frame_Eh'].flatten()))
                # raw_data.extend((np.angle(mat_data['frame_Eh']).flatten()))
                raw_data.extend(((np.angle(mat_data['frame_Eh'].flatten()) / np.pi + 1) * 100))
                # print(Angle(mat_data['frame_Ev'].flatten()))
                # break
            else:
                raw_data.extend(np.abs(mat_data['frame_Ev']).flatten())
                raw_data.extend(np.abs(mat_data['frame_Eh']).flatten())
            flat_data.append(raw_data)
            # images.append(img)
            target.append(categories[i])
        # break
    # return [flat_data,target,categories,descr]
    flat_data = np.array(flat_data)
    target = np.array(target)
    return Bunch(data=flat_data,
                 target=target,
                 target_names=categories,
                 DESCR=descr)


def LOAD_IMAGE():
    image_dataset1 = load_image_files('.\dataRCS')
    image_dataset2 = load_image_files('.\dataRCS2')
    # flat_data = np.concatenate([image_dataset1.data, image_dataset2.data])
    # target = np.concatenate([image_dataset1.target, image_dataset2.target])
    # categories = np.concatenate([image_dataset1.categories, image_dataset2.categories])
    # descr = image_dataset1.descr
    categories = image_dataset1.target_names
    categories.extend(image_dataset2.target_names)
    print("fucked")
    return Bunch(data=np.concatenate([image_dataset1.data, image_dataset2.data]),
                 target=np.concatenate([image_dataset1.target, image_dataset2.target]),
                 target_names=categories,
                 DESCR=image_dataset1.DESCR)


# ----KC501-----
# image_dataset = load_image_files('/home/kc501/LJY/Alexnet/dataRCS')

# ----ICraft----
t_load_data = time()
image_dataset = LOAD_IMAGE()
# image_dataset1 = load_image_files('.\dataRCS')
# image_dataset2 = load_image_files('.\dataRCS2')
# # flat_data = np.concatenate([image_dataset1.data, image_dataset2.data])
# # target = np.concatenate([image_dataset1.target, image_dataset2.target])
# # categories = np.concatenate([image_dataset1.categories, image_dataset2.categories])
# # descr = image_dataset1.descr
# categories = image_dataset1.target_names
# categories.extend(image_dataset2.target_names)
# print("fucked")
# image_dataset = Bunch(data=np.concatenate([image_dataset1.data, image_dataset2.data]),
#                       target=np.concatenate([image_dataset1.target, image_dataset2.target]),
#                       target_names=categories,
#                       DESCR=image_dataset1.DESCR)
print("Data load done in %0.3fs" % (time() - t_load_data))
# image_dataset_test = load_image_files("E:/RL_code/alex-net-image-classification-master/class3/val")

0 1
1 2
2 3
3 4
4 5
0 10
1 6
2 7
3 8
4 9
fucked
Data load done in 204.128s


In [3]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(
    image_dataset.data, image_dataset.target, test_size=0.2, random_state=101)

In [ ]:
n_components = 100
tolerance = 1e-2
max_iteration = 10000
if not NO_NMF:
    t_NMF = time()
    nmf = NMF(n_components=n_components, init='nndsvd', tol=tolerance, max_iter=max_iteration, verbose=True).fit(
        X_train)
    # W = nmf.components_.reshape((n_components, 64, 64))

    X_train_nmf = nmf.transform(X_train)
    X_test_nmf = nmf.transform(X_test)
    print("NMF done in %0.3fs" % (time() - t_NMF))

violation: 1.0
violation: 0.415232152475027
violation: 0.328275720935333
violation: 0.2951128614168521
violation: 0.277334130957208
violation: 0.26375973425707244
violation: 0.25296562881321866
violation: 0.24296488353214923
violation: 0.23429389003810752
violation: 0.22590803163311945
violation: 0.2197988908177308
violation: 0.2123233367838285
violation: 0.2067661630929283
violation: 0.20138332464104772
violation: 0.1965566504981698
violation: 0.1921939186915778
violation: 0.18907546876511871
violation: 0.18585473916377004
violation: 0.18192699880398835
violation: 0.1783070432938446
violation: 0.17511291781779498
violation: 0.17186517096743695
violation: 0.16868705698433914
violation: 0.16572006188846636
violation: 0.1627726645838626
violation: 0.1595327704458579


In [25]:
f = open('./TZB.txt', 'a')
t_train = time()
kernel = 'poly'
clf = SVC(C=1, kernel=kernel, gamma=0.01, class_weight='balanced', decision_function_shape='ovo', verbose=True)
print("SVM done in %0.3fs" % (time() - t_train))

if NO_NMF:
    clf = clf.fit(X_train, y_train)
    y_pred_train = clf.predict(X_train)
    y_pred_test = clf.predict(X_test)
else:
    clf = clf.fit(X_train_nmf, y_train)
    y_pred_train = clf.predict(X_train_nmf)
    y_pred_test = clf.predict(X_test_nmf)

y_true_train = y_train
y_true_test = y_test

f.write("n_components: {}\n".format(n_components))
f.write("kernel: {}\n".format(kernel))
f.write("tolerance: {}\n".format(tolerance))
f.write("max_iteration: {}\n".format(max_iteration))
f.write("Acc on train data: {}\n".format(accuracy_score(y_true_train, y_pred_train)))
f.write("Acc on test data: {}\n".format(accuracy_score(y_true_test, y_pred_test)))
f.write("------------------------------------------------------------------------------\n")
f.close()
print("n_components: ", n_components)
print("Acc on train data:" + str(accuracy_score(y_true_train, y_pred_train)))

print("Acc on test data:" + str(accuracy_score(y_true_test, y_pred_test)))

SVM done in 0.000s
[LibSVM]n_components:  20
Acc on train data:0.9585
Acc on test data:0.238
